In [1]:
import torch 
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import glob 
import re
from tkinter import E
csv_files = glob.glob('C:/Users/pablo/Desktop/scripts tfg/archivos min/*.min')

# Create an empty dataframe to store the combined data
combined_df_min = pd.DataFrame()
ind=['min HFB','min PNP','min PNPAMP']
combined_df_min.index=ind
E_HFB=[]
E_PNPAMP=[]
list_zn=[]
# Loop through each CSV file and append its contents to the combined dataframe
for csv_file in csv_files:
    df = pd.read_csv(csv_file,sep=' ',header=None)
    df.columns=['beta_2','E HFB','E PNP', 'E PNPAMP']
    zn=[int(s) for s in re.findall(r'\d+', csv_file)] 
    list_zn.append(zn)
    E_HFB.append(np.abs(df.iloc[0,1]))
    E_PNPAMP.append(np.abs(df.iloc[2,3]))


In [30]:
class MyModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MyModel, self).__init__()
        self.dropout = nn.Dropout(p=0.3)
        self.fc1 = nn.Linear(input_dim, 1000)
        self.fc2 = nn.Linear(1000, 750) 
        self.fc3 = nn.Linear(750,600)
        self.fc4 = nn.Linear (600, 800)
        self.fc5 = nn.Linear(800, output_dim)
        self.internal = nn.LeakyReLU()
        self.activation = nn.ReLU()
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.zeros_(self.fc1.bias)        
        self.batchnorm1=nn.BatchNorm1d(1000, eps=1e-05, momentum=None)#batch norm should be frozen when evaluating the model
        self.batchnorm2=nn.BatchNorm1d(750, eps=1e-05, momentum=None)
        self.batchnorm3=nn.BatchNorm1d(600, eps=1e-05, momentum=None)
        self.batchnorm4=nn.BatchNorm1d(800, eps=1e-05, momentum=None)

    def forward(self, x):
        x1 = self.internal(self.fc1(x))
        x1= self.batchnorm1(x1)
        x1= self.dropout(x1)
        x2 = self.internal(self.fc2(x1))
        x2=self.batchnorm2(x2)
        x2= self.dropout(x2)
        x3 = self.internal(self.fc3(x2))
        x3=self.batchnorm3(x3)
        x3= self.dropout(x3)
        x4 = self.internal(self.fc4(x3))
        x4=self.batchnorm4(x4)
        x4= self.dropout(x4)
        y=self.activation(self.fc5(x4))
        return y 


In [12]:
list_zn_E=[]
i=0
for z, n in list_zn: 
    #list_zn_E.append(n)
    #list_zn_E.append(z)
    list_zn_E.append([z,n, E_HFB[i]])
    i+=1

print(list_zn_E)


[[10, 8, 134.13283577], [10, 10, 156.7515792], [10, 12, 175.5828406], [10, 14, 189.26900267], [10, 16, 199.91119992], [10, 18, 205.64333384], [10, 20, 209.78941438], [10, 22, 209.87037237], [10, 24, 209.83603783], [12, 8, 136.52084595], [12, 10, 167.23401557], [12, 12, 193.93228165], [12, 14, 212.99280104], [12, 16, 228.90796101], [12, 18, 239.0154746], [12, 20, 247.90651214], [12, 22, 253.05758245], [12, 24, 257.53664193], [12, 26, 259.18361774], [12, 28, 259.4021756], [14, 10, 171.22694835], [14, 12, 203.55039909], [14, 14, 233.15506359], [14, 16, 252.41185084], [14, 18, 268.48480185], [14, 20, 282.18347819], [14, 22, 289.62970388], [14, 24, 296.79004759], [14, 26, 303.01087952], [14, 28, 308.95765151], [14, 30, 309.67642715], [14, 32, 309.84466925], [14, 34, 308.39872702], [16, 10, 171.00457191], [16, 12, 207.96184782], [16, 14, 240.90787203], [16, 16, 267.5076118], [16, 18, 288.11769782], [16, 20, 306.81580912], [16, 22, 318.91990047], [16, 24, 331.18463043], [16, 26, 341.33604053]

In [13]:
x=torch.tensor(list_zn_E).float()
y=torch.tensor(E_PNPAMP).float()


In [14]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(x, y)
split_ratio= 0.5
train_size = int(len(dataset) * split_ratio)
test_size = len(dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset,batch_size=train_size, shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=test_size, shuffle=False)

In [31]:
net_slice=MyModel(3,1)
net_slice=net_slice.float()
criterion = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(net_slice.parameters(), lr=1e-3)
for nepoch in range(2000):
    for x_train_slice, y_train_slice in train_loader:
        hatY_slice = net_slice.forward(x_train_slice)
        optimizer.zero_grad()
        loss_slice = criterion(hatY_slice, y_train_slice)
        loss_slice.backward()
        optimizer.step()
        # solo sobre el test    
        #acc=np.std()#loss segun tamaño entreno y test y varianza
        #entrenar sobre las propias energías
        #hablar sobre tiempos de computación
        #rel_acc=acc/y_test.mean()
        if nepoch % 100 == 0:
            print(f"loss for {nepoch} = {loss_slice.item():0.2f}")
for x_test_slice, y_test_slice in test_loader:
    hatY_pred_slice=net_slice.forward(x_test_slice)
    hatY_detach_slice=hatY_pred_slice.detach()
    total_loss_slice=criterion(hatY_detach_slice,y_test_slice)
    print(total_loss_slice.item())

loss for 0 = 2026976.25
loss for 100 = 1872966.75
loss for 200 = 1671429.88
loss for 300 = 1422080.50
loss for 400 = 1158832.12
loss for 500 = 919711.50
loss for 600 = 724827.19
loss for 700 = 573801.75
loss for 800 = 461302.84
loss for 900 = 395040.81
loss for 1000 = 366784.94
loss for 1100 = 333723.12
loss for 1200 = 295244.84
loss for 1300 = 273725.78
loss for 1400 = 262514.47
loss for 1500 = 249051.12
loss for 1600 = 248873.17
loss for 1700 = 242819.88
loss for 1800 = 241282.45
loss for 1900 = 240725.86
215446.265625


In [34]:
class MyModel_norm(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MyModel_norm, self).__init__()
        #self.dropout = nn.Dropout(p=0.3)
        self.fc1 = nn.Linear(input_dim, 1000)
        self.fc2 = nn.Linear(1000, 750) 
        self.fc3 = nn.Linear(750,600)
        self.fc4 = nn.Linear (600, 800)
        self.fc5 = nn.Linear(800, output_dim)
        self.internal = nn.LeakyReLU()
        self.activation = nn.ReLU()
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.zeros_(self.fc1.bias)        
        self.batchnorm1=nn.BatchNorm1d(1000, eps=1e-05, momentum=None)#batch norm should be frozen when evaluating the model
        self.batchnorm2=nn.BatchNorm1d(750, eps=1e-05, momentum=None)
        self.batchnorm3=nn.BatchNorm1d(600, eps=1e-05, momentum=None)
        self.batchnorm4=nn.BatchNorm1d(800, eps=1e-05, momentum=None)

    def forward(self, x):
        x1 = self.internal(self.fc1(x))
        x1= self.batchnorm1(x1)
        #x1= self.dropout(x1)
        x2 = self.internal(self.fc2(x1))
        x2=self.batchnorm2(x2)
        #x2= self.dropout(x2)
        x3 = self.internal(self.fc3(x2))
        x3=self.batchnorm3(x3)
        #x3= self.dropout(x3)
        x4 = self.internal(self.fc4(x3))
        x4=self.batchnorm4(x4)
        #x4= self.dropout(x4)
        y=self.activation(self.fc5(x4))
        return y 


In [35]:
net_slice=MyModel_norm(3,1)
net_slice=net_slice.float()
criterion = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(net_slice.parameters(), lr=1e-3)
for nepoch in range(2000):
    for x_train_slice, y_train_slice in train_loader:
        hatY_slice = net_slice.forward(x_train_slice)
        optimizer.zero_grad()
        loss_slice = criterion(hatY_slice, y_train_slice)
        loss_slice.backward()
        optimizer.step()
        # solo sobre el test    
        #acc=np.std()#loss segun tamaño entreno y test y varianza
        #entrenar sobre las propias energías
        #hablar sobre tiempos de computación
        #rel_acc=acc/y_test.mean()
        if nepoch % 100 == 0:
            print(f"loss for {nepoch} = {loss_slice.item():0.2f}")
for x_test_slice, y_test_slice in test_loader:
    hatY_pred_slice=net_slice.forward(x_test_slice)
    hatY_detach_slice=hatY_pred_slice.detach()
    total_loss_slice=criterion(hatY_detach_slice,y_test_slice)
    print(total_loss_slice.item())

c:\Users\pablo\AppData\Local\Programs\Python\Python37\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([856])) that is different to the input size (torch.Size([856, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss for 0 = 2026972.25
loss for 100 = 1873825.50
loss for 200 = 1684388.00
loss for 300 = 1431423.62
loss for 400 = 1159783.62
loss for 500 = 909358.69
loss for 600 = 701898.75
loss for 700 = 579732.81
loss for 800 = 522456.22
loss for 900 = 501391.81
loss for 1000 = 495306.44
loss for 1100 = 493883.50
loss for 1200 = 493614.34
loss for 1300 = 493575.50
loss for 1400 = 493565.75
loss for 1500 = 493565.12
loss for 1600 = 493565.06
loss for 1700 = 493565.06
loss for 1800 = 493565.12
loss for 1900 = 493565.06
474026.125
